notebook that checks if entries are valid


In [1]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession, functions as F
import os
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/20 13:06:04 WARN Utils: Your hostname, DESKTOP-RBVA59Q resolves to a loopback address: 127.0.1.1; using 172.26.88.196 instead (on interface eth0)
24/09/20 13:06:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/20 13:06:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet('../data/raw/property_data.parquet')

In [4]:
df.show(5)

+--------------------+--------+---------------+--------------------+--------------------+----+-----+-------+--------------------+
|                 url|postcode|         suburb|                name|           cost_text|beds|baths|parking|       property_type|
+--------------------+--------+---------------+--------------------+--------------------+----+-----+-------+--------------------+
|https://www.domai...|    3141|    south-yarra|(Leased) 3 Yarra ...|                $460|   1|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3125|        burwood|04/390 Burwood Hi...|$310 per week, wi...|   1|    1|      0|Apartment / Unit ...|
|https://www.domai...|    3156| ferntree-gully|1 & 2/32 Folkston...|                $800|   4|    3|      1|               House|
|https://www.domai...|    3228|        torquay|1 Acacia Street, ...|     $1,000 per week|   4|    2|      2|               House|
|https://www.domai...|    3200|frankston-north|1 Aleppo Crescent...|                $450| 

In [5]:
small_df= df.limit(20)

In [43]:

updated_df.orderBy(F.desc("parking"))

url,postcode,suburb,name,cost_text,beds,baths,parking,property_type,contains_price
https://www.domai...,3753,beveridge,29 Stewart Street...,$750,3,2,6,House,true
https://www.domai...,3190,highett,"60 Graham Road, H...","$1,500.00",5,4,6,House,true
https://www.domai...,3608,kirwans-bridge,"31 Backwater Ct, ...",$530,3,2,6,House,true
https://www.domai...,3219,whittington,"169 Solar Drive, ...",$470,3,1,6,House,true
https://www.domai...,3128,box-hill,"484 Elgar Road, B...",APPLY,1,1,6,Apartment / Unit ...,true
https://www.domai...,3150,wheelers-hill,"34 Darnley Grove,...",$630,4,2,6,House,true
https://www.domai...,3128,box-hill,"490 Elgar Road, B...",APPLY,1,1,6,Apartment / Unit ...,true
https://www.domai...,3930,mount-eliza,"77 Walkers Road, ...","$1,300.00",4,2,6,House,true
https://www.domai...,3202,heatherton,10-12 Pietro Road...,"$1,250",4,2,6,House,true
https://www.domai...,3815,tonimbuk,Tonimbuk VIC 3815,$1000.00,3,2,6,Acreage / Semi-Rural,true


In [6]:
small_df.show()

+--------------------+--------+---------------+--------------------+--------------------+----+-----+-------+--------------------+
|                 url|postcode|         suburb|                name|           cost_text|beds|baths|parking|       property_type|
+--------------------+--------+---------------+--------------------+--------------------+----+-----+-------+--------------------+
|https://www.domai...|    3141|    south-yarra|(Leased) 3 Yarra ...|                $460|   1|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3125|        burwood|04/390 Burwood Hi...|$310 per week, wi...|   1|    1|      0|Apartment / Unit ...|
|https://www.domai...|    3156| ferntree-gully|1 & 2/32 Folkston...|                $800|   4|    3|      1|               House|
|https://www.domai...|    3228|        torquay|1 Acacia Street, ...|     $1,000 per week|   4|    2|      2|               House|
|https://www.domai...|    3200|frankston-north|1 Aleppo Crescent...|                $450| 

In [38]:
updated_df = df \
    .filter((F.col('beds') > 0) & (F.col('beds') <= 6)) \
    .filter((F.col('baths') > 0) & (F.col('baths') <= 6)) \
    .filter((F.col('parking') >= 0) & (F.col('parking') <= 6)) \

updated_df = updated_df.withColumn(
    "contains_price",
    F.col("cost_text").rlike(r"(\$*\d+(?:,\d*)*(?:\.\d*)?)")
).filter(F.col("contains_price") == True)

In [39]:
updated_df.orderBy(F.desc("beds"))

url,postcode,suburb,name,cost_text,beds,baths,parking,property_type,contains_price
https://www.domai...,3183,st-kilda-east,242 Dandenong Roa...,"$1,495",6,2,1,House,true
https://www.domai...,3184,elwood,22 Dickens Street...,"$1,195",6,2,1,House,true
https://www.domai...,3227,barwon-heads,26 Riverside Terr...,"$1,800.00",6,2,4,House,true
https://www.domai...,3226,ocean-grove,"160 Shell Road, O...",$680.00,6,3,2,House,true
https://www.domai...,3939,boneo,451 Truemans Road...,"$2,200.00",6,5,3,House,true
https://www.domai...,3150,glen-waverley,13 Stirling Cresc...,"$1,100",6,4,2,House,true
https://www.domai...,3978,clyde-north,47 Littleshore Cr...,$900,6,3,2,House,true
https://www.domai...,3029,tarneit,66 Mulholland Dri...,$850,6,3,2,House,true
https://www.domai...,3787,sassafras,"2 Aurora Rise, Sa...",$2800,6,6,6,House,true
https://www.domai...,3187,brighton-east,"8 Beltane Avenue,...","$1,050.00",6,4,2,House,true


In [34]:
spec = updated_df.filter(updated_df.suburb == "cranbourne-east").orderBy(F.desc("beds"))

In [35]:
spec.show()

+--------------------+--------+---------------+--------------------+--------------------+----+-----+-------+-------------+--------------+
|                 url|postcode|         suburb|                name|           cost_text|beds|baths|parking|property_type|contains_price|
+--------------------+--------+---------------+--------------------+--------------------+----+-----+-------+-------------+--------------+
|https://www.domai...|    3977|cranbourne-east|1 Camaro Drive, C...|  $680pw | $2,955pcm|   6|    2|      2|        House|          true|
|https://www.domai...|    3977|cranbourne-east|15 Guineas Street...|                $650|   4|    2|      2|        House|          true|
|https://www.domai...|    3977|cranbourne-east|20 Challenger Cir...|  $610pw | $2,651pcm|   4|    2|      2|        House|          true|
|https://www.domai...|    3977|cranbourne-east|30 Collison Road,...|$1,000pw | $4,346pcm|   4|    2|      4|        House|          true|
|https://www.domai...|    3977|cra

In [37]:
spec.withColumn(
    'cost_text',
    F.split(F.col('cost_text'), " ").getItem(0)
)

url,postcode,suburb,name,cost_text,beds,baths,parking,property_type,contains_price
https://www.domai...,3977,cranbourne-east,"1 Camaro Drive, C...",$680pw,6,2,2,House,true
https://www.domai...,3977,cranbourne-east,15 Guineas Street...,$650,4,2,2,House,true
https://www.domai...,3977,cranbourne-east,20 Challenger Cir...,$610pw,4,2,2,House,true
https://www.domai...,3977,cranbourne-east,"30 Collison Road,...","$1,000pw",4,2,4,House,true
https://www.domai...,3977,cranbourne-east,43 Guineas Street...,$540pw,4,2,2,House,true
https://www.domai...,3977,cranbourne-east,"6 Ogilvie Court, ...",$625,4,2,2,House,true
https://www.domai...,3977,cranbourne-east,"8 Somers Street, ...",$580,4,2,2,House,true
https://www.domai...,3977,cranbourne-east,9 Mossey Crescent...,$520,4,2,1,House,true
https://www.domai...,3977,cranbourne-east,"25 Reynard Place,...",$650,4,2,2,House,true
https://www.domai...,3977,cranbourne-east,"6 Pony Court, Cra...",$580,4,2,2,House,true


In [16]:
print("removing {} invalid rows".format(df.count() - updated_df.count()))


removing 198 invalid rows


In [17]:
updated_df.write.mode("overwrite").parquet("../data/curated/valid_df.parquet")

In [ ]:
# creating a function for this task
def find_valid(df: SparkSession.DataFrame , output_path: str) -> None:
    """Function that filters for rows that are valid and outputs into a parquet file.
    A row is deemed valid if:
                    - beds : (0,6]
                    - baths : (0,6]
                    - parking : [0, 6]
    The justification for these values is based on looking at the data and deciding that prices with properties which state these amounts of beds, baths and parking are far too cheap
    
    Parameters:
    df - Dataframe which we want to find valid rows for
    output_path - string that brings us to our desired output path

    """
    updated_df = df \
    .filter((F.col('beds') > 0) & (F.col('beds') <= 6)) \
    .filter((F.col('baths') > 0) & (F.col('baths') <= 6)) \
    .filter((F.col('parking') >= 0) & (F.col('parking') <= 6)) \

    updated_df = updated_df.withColumn(
        "contains_price",
        F.col("cost_text").rlike(r"(\$*\d+(?:,\d*)*(?:\.\d*)?)")
    ).filter(F.col("contains_price") == True).drop(F.col("contains_price"))

    print("removing {} invalid rows".format(df.count() - updated_df.count()))
    # write to file
    updated_df.write.mode("overwrite").parquet(output_path)
